### Import libraries and set the height, width and channels of image.

In [1]:
import keras
import tensorflow
from keras import layers
import numpy as np
latent_dim = 32
height = 32
width = 32
channels = 3

### Define the Generator

In [2]:
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense (Dense)               (None, 32768)             1081344   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32768)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 128)       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 256)      104883

### Define the Discriminator

In [3]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(
 lr=0.0008,
 clipvalue=1.0,
 decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 30, 30, 128)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 6, 6, 128)         0   

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### Define the GAN model where the discriminator is not trainable.
#### This model turns latent-space points into a classification
####decision—“fake” or “real”—and it’s meant to be trained with labels that are always “these are real images.”

In [4]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

### Download the cifar dataset

In [5]:
import os
from keras.preprocessing import image
(x_train, y_train), (_, _) = tensorflow.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


### We are only concerned with the 'frog' class which is labeled 6

In [6]:
x_train = x_train[y_train.flatten() == 6]

In [7]:
x_train = x_train.reshape(
(x_train.shape[0],) +
(height, width, channels)).astype('float32') / 255

### Set the iterations and directory to which generated images can be downloaded

In [8]:
iterations = 2000
batch_size = 20
save_dir = '/content/'

In [9]:
from tensorflow.keras.preprocessing import image

#### First train the discriminator with real images as well as fake ones generated by generator. After that disable discriminator training and train the GAN model with generated images, all labeled as 'real'.

In [10]:
start = 0
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  generated_images = generator.predict(random_latent_vectors)

  stop = start + batch_size
  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])

  labels = np.concatenate([np.ones((batch_size, 1)),
  np.zeros((batch_size, 1))])
  labels += 0.05 * np.random.random(labels.shape)

  d_loss = discriminator.train_on_batch(combined_images, labels)

  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

  misleading_targets = np.zeros((batch_size, 1))
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if step % 100 == 0:
    gan.save_weights('gan.h5')

    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'real_frog' + str(step) + '.png'))



1/1 [==============================] - 10s 10s/step
discriminator loss: 0.6996825337409973
adversarial loss: 0.687693178653717
1/1 [==============================] - 0s 26ms/step
discriminator loss: 0.4889760911464691
adversarial loss: 2.0340373516082764
1/1 [==============================] - 0s 30ms/step
discriminator loss: 0.6765270829200745
adversarial loss: 1.1556813716888428
1/1 [==============================] - 0s 19ms/step
discriminator loss: 0.6819178462028503
adversarial loss: 0.7678629159927368
1/1 [==============================] - 0s 20ms/step
discriminator loss: 0.6922010779380798
adversarial loss: 0.7387372851371765
1/1 [==============================] - 0s 17ms/step
discriminator loss: 0.7037619352340698
adversarial loss: 0.740537166595459
1/1 [==============================] - 0s 19ms/step
discriminator loss: 0.689741313457489
adversarial loss: 0.7511263489723206
1/1 [==============================] - 0s 33ms/step
discriminator loss: 0.6871262788772583
adversarial loss